# Dog venues in Boston, MA

In [288]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
!conda install -c conda-forge geopy --yes
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


Solving environment: ...working... done

# All requested packages already installed.



In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [101]:
CLIENT_ID = 'NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM' # your Foursquare ID
CLIENT_SECRET = 'CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT' # your Foursquare Secret
ACCESS_TOKEN = 'NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('access' + ACCESS_TOKEN)

Your credentails:
CLIENT_ID: NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM
CLIENT_SECRET:CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT
accessNYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX


#### Getting coordinates for Boston, MA

In [174]:
address = 'Boston, MA'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

42.3602534 -71.0582912


#### Searching for entries containing 'Park' near Boston, MA

In [175]:
# SEARCHING FOR PARKS NEAR BOSTON

search_query = ['Park']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d163941735&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?categoryId=4bf58dd8d48988d163941735&client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Park']&radius=7000&limit=500"

In [176]:
# GETTING THE JSON FILE

results_park = requests.get(url).json()
results_park

{'meta': {'code': 200, 'requestId': '601f3fd011e9d93e69f7a73e'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ba50617f964a520f1d138e3',
    'name': 'North End Park',
    'location': {'address': 'Cross St.',
     'crossStreet': 'btwn North & Sudbury',
     'lat': 42.36248823184806,
     'lng': -71.05647696102726,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.36248823184806,
       'lng': -71.05647696102726}],
     'distance': 290,
     'postalCode': '02113',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Cross St. (btwn North & Sudbury)',
      'Boston, MA 02113']},
    'categories': [{'id': '4bf58dd8d48988d163941735',
      'name': 'Park',
      'pluralName': 'Parks',
      'shortName': 'Park',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
       'suffix': '.png'},
      'primary': True}],
 

#### Creating dataframe

In [235]:
# assign relevant part of JSON to venues
venues_park = results_park['response']['venues']

# tranform venues into a dataframe
dataframe_park = json_normalize(venues_park)
dataframe_park.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4ba50617f964a520f1d138e3,Cross St.,US,Boston,United States,btwn North & Sudbury,290,"[Cross St. (btwn North & Sudbury), Boston, MA ...","[{'label': 'display', 'lat': 42.36248823184806...",42.362488,-71.056477,NaN,02113,MA,North End Park,v-1612660688,NaN
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4ae3580af964a520c59321e3,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,"[Atlantic Ave. (at Atlantic Ave.), Boston, MA ...","[{'label': 'display', 'lat': 42.36102715769776...",42.361027,-71.051451,NaN,02110,MA,Christopher Columbus Park,v-1612660688,NaN
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,54a1d1ce498e248911805297,NaN,US,Boston,United States,NaN,448,"[Boston, MA]","[{'label': 'display', 'lat': 42.36072318013166...",42.360723,-71.052873,NaN,NaN,MA,Harbor Park,v-1612660688,NaN
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bcf72669854d13a8f13f64d,NaN,US,Charlestown,United States,Chelsea St,1143,"[Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37019699121477...",42.370197,-71.061781,NaN,02129,MA,Paul Revere Park,v-1612660688,NaN
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",False,4bd9bd3fd2cbc92844a3d1ad,Museum Way,US,Cambridge,United States,NaN,1332,"[Museum Way, Cambridge, MA 02141]","[{'label': 'display', 'lat': 42.36911761365681...",42.369118,-71.069175,NaN,02141,MA,North Point Park,v-1612660688,NaN


#### Cleaning up dataframe

In [236]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_park = ['name', 'categories'] + [col for col in dataframe_park.columns if col.startswith('location.')] + ['id']
dataframe_filtered_park = dataframe_park.loc[:, filtered_columns_park]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_park['categories'] = dataframe_filtered_park.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_park.columns = [column.split('.')[-1] for column in dataframe_filtered_park.columns]

dataframe_filtered_park.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,"[Cross St. (btwn North & Sudbury), Boston, MA ...","[{'label': 'display', 'lat': 42.36248823184806...",42.362488,-71.056477,NaN,02113,MA,4ba50617f964a520f1d138e3
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,"[Atlantic Ave. (at Atlantic Ave.), Boston, MA ...","[{'label': 'display', 'lat': 42.36102715769776...",42.361027,-71.051451,NaN,02110,MA,4ae3580af964a520c59321e3
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,"[Boston, MA]","[{'label': 'display', 'lat': 42.36072318013166...",42.360723,-71.052873,NaN,NaN,MA,54a1d1ce498e248911805297
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,"[Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37019699121477...",42.370197,-71.061781,NaN,02129,MA,4bcf72669854d13a8f13f64d
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,"[Museum Way, Cambridge, MA 02141]","[{'label': 'display', 'lat': 42.36911761365681...",42.369118,-71.069175,NaN,02141,MA,4bd9bd3fd2cbc92844a3d1ad


#### Map of parks

In [180]:
venues_map_park = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_park.lat, dataframe_filtered_park.lng, dataframe_filtered_park.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_park)

# display map
venues_map_park

#### Searching for entries containing 'pet'

In [182]:
# SEARCHING FOR PETS NEAR BOSTON

search_query = ['Pet']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Pet']&radius=7000&limit=500"

In [183]:
# GETTING THE JSON FILE

results_pet = requests.get(url).json()
results_pet

{'meta': {'code': 200, 'requestId': '601f404721d85f1b7acf7102'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '5f84ad1ab70817260c693afc',
    'name': 'The Pet Vet - North End',
    'location': {'lat': 42.364488,
     'lng': -71.060868,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.364488,
       'lng': -71.060868}],
     'distance': 516,
     'postalCode': '02114',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Boston, MA 02114']},
    'categories': [{'id': '4d954af4a243a5684765b473',
      'name': 'Veterinarian',
      'pluralName': 'Veterinarians',
      'shortName': 'Veterinarians',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_veterinarian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1612660808',
    'hasPerk': False},
   {'id': '56045e14498e6d37759308a3',
    'name

#### Create a dataframe

In [184]:
# assign relevant part of JSON to venues
venues_pet = results_pet['response']['venues']

# tranform venues into a dataframe
dataframe_pet = json_normalize(venues_pet)
dataframe_pet.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,5f84ad1ab70817260c693afc,NaN,US,Boston,United States,NaN,516,"[Boston, MA 02114]","[{'label': 'display', 'lat': 42.364488, 'lng':...",42.364488,-71.060868,02114,MA,The Pet Vet - North End,v-1612660808,NaN
1,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,56045e14498e6d37759308a3,630 Fellsway,US,Medford,United States,NaN,5577,"[630 Fellsway, Medford, MA 02155]","[{'label': 'display', 'lat': 42.40683108115126...",42.406831,-71.083285,02155,MA,Pet Supplies Plus Medford,v-1612660808,NaN
2,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",NaN,NaN,NaN,NaN,NaN,NaN,False,5697e553498e21a5b27ed54f,5 Mystic View Rd,US,Everett,United States,NaN,4360,"[5 Mystic View Rd, Everett, MA 02149]","[{'label': 'display', 'lat': 42.398629, 'lng':...",42.398629,-71.068905,02149,MA,Banfield Pet Hospital,v-1612660808,NaN
3,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbf859fabf49521f1c8bd93,274 Southampton St,US,Boston,United States,NaN,3270,"[274 Southampton St, Boston, MA 02118]","[{'label': 'display', 'lat': 42.33245215, 'lng...",42.332452,-71.071146,02118,MA,Skipton Pet Center,v-1612660808,NaN
4,[],NaN,NaN,NaN,NaN,NaN,NaN,False,54cf294b498e696f2488e595,NaN,US,Boston,United States,NaN,52,"[Boston, MA]","[{'label': 'display', 'lat': 42.3600825, 'lng'...",42.360082,-71.058880,NaN,MA,Brandy Pete's,v-1612660808,NaN


In [185]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_pet = ['name', 'categories'] + [col for col in dataframe_pet.columns if col.startswith('location.')] + ['id']
dataframe_filtered_pet = dataframe_pet.loc[:, filtered_columns_pet]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_pet['categories'] = dataframe_filtered_pet.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_pet.columns = [column.split('.')[-1] for column in dataframe_filtered_pet.columns]

dataframe_filtered_pet.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,The Pet Vet - North End,Veterinarian,NaN,US,Boston,United States,NaN,516,"[Boston, MA 02114]","[{'label': 'display', 'lat': 42.364488, 'lng':...",42.364488,-71.060868,02114,MA,5f84ad1ab70817260c693afc
1,Pet Supplies Plus Medford,Pet Store,630 Fellsway,US,Medford,United States,NaN,5577,"[630 Fellsway, Medford, MA 02155]","[{'label': 'display', 'lat': 42.40683108115126...",42.406831,-71.083285,02155,MA,56045e14498e6d37759308a3
2,Banfield Pet Hospital,Veterinarian,5 Mystic View Rd,US,Everett,United States,NaN,4360,"[5 Mystic View Rd, Everett, MA 02149]","[{'label': 'display', 'lat': 42.398629, 'lng':...",42.398629,-71.068905,02149,MA,5697e553498e21a5b27ed54f
3,Skipton Pet Center,Pet Store,274 Southampton St,US,Boston,United States,NaN,3270,"[274 Southampton St, Boston, MA 02118]","[{'label': 'display', 'lat': 42.33245215, 'lng...",42.332452,-71.071146,02118,MA,4bbf859fabf49521f1c8bd93
4,Brandy Pete's,None,NaN,US,Boston,United States,NaN,52,"[Boston, MA]","[{'label': 'display', 'lat': 42.3600825, 'lng'...",42.360082,-71.058880,NaN,MA,54cf294b498e696f2488e595


#### Create map for venues

In [326]:
venues_map_pet = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_pet.lat, dataframe_filtered_pet.lng, dataframe_filtered_pet.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_pet)

# display map
venues_map_pet

#### Search for entries containing 'dog'

In [188]:
# SEARCHING FOR DOG NEAR BOSTON

search_query = ['Dog']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Dog']&radius=7000&limit=500"

In [189]:
# GETTING THE JSON FILE

results_dog = requests.get(url).json()
results_dog

{'meta': {'code': 200, 'requestId': '601f409e55b9505d04a07175'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '40b28c80f964a520b5f71ee3',
    'name': 'Salty Dog Seafood Grille & Bar',
    'location': {'address': '206 Faneuil Hall Market Pl',
     'lat': 42.35993714563105,
     'lng': -71.05567659306708,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.35993714563105,
       'lng': -71.05567659306708}],
     'distance': 217,
     'postalCode': '02109',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['206 Faneuil Hall Market Pl', 'Boston, MA 02109']},
    'categories': [{'id': '4bf58dd8d48988d1ce941735',
      'name': 'Seafood Restaurant',
      'pluralName': 'Seafood Restaurants',
      'shortName': 'Seafood',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/seafood_',
       'suffix': '.png'},
      'primary': True}],


#### Create a dataframe

In [190]:
# assign relevant part of JSON to venues
venues_dog = results_dog['response']['venues']

# tranform venues into a dataframe
dataframe_dog = json_normalize(venues_dog)
dataframe_dog.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,40b28c80f964a520b5f71ee3,206 Faneuil Hall Market Pl,US,Boston,United States,NaN,217,"[206 Faneuil Hall Market Pl, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35993714563105...",42.359937,-71.055677,02109,MA,Salty Dog Seafood Grille & Bar,v-1612660894,NaN
1,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,4c335e83ac0ab7136bbb1c1e,1 Central Wharf Boston,US,Boston,United States,NaN,730,"[1 Central Wharf Boston, Boston, MA 02110]","[{'label': 'display', 'lat': 42.35917282104492...",42.359173,-71.049538,02110,MA,Dog n' Claw,v-1612660894,NaN
2,"[{'id': '5453de49498eade8af355881', 'name': 'B...",False,59362c81d0a14958ac92bd26,8 Faneuil Hall Market Pl Fl 3,US,Boston,United States,NaN,62,"[8 Faneuil Hall Market Pl Fl 3, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35988214932819...",42.359882,-71.058862,02109,MA,My Busy Dog,v-1612660894,NaN
3,"[{'id': '4bf58dd8d48988d1e5941735', 'name': 'D...",False,4bc7176114d79521e3ad66e9,1277 Washington Street,US,Boston,United States,East Berkeley,2104,"[1277 Washington Street (East Berkeley), Bosto...","[{'label': 'display', 'lat': 42.34269554511267...",42.342696,-71.067776,02118,MA,Joe Wex Dog Recreation Space,v-1612660894,NaN
4,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",False,585c1342d98f36440558fe33,NaN,US,Boston,United States,NaN,363,"[Boston, MA]","[{'label': 'display', 'lat': 42.363006, 'lng':...",42.363006,-71.055918,NaN,MA,Polka Dog Bakery,v-1612660894,NaN


In [191]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_dog = ['name', 'categories'] + [col for col in dataframe_dog.columns if col.startswith('location.')] + ['id']
dataframe_filtered_dog = dataframe_dog.loc[:, filtered_columns_dog]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_dog['categories'] = dataframe_filtered_dog.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_dog.columns = [column.split('.')[-1] for column in dataframe_filtered_dog.columns]

dataframe_filtered_dog.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Salty Dog Seafood Grille & Bar,Seafood Restaurant,206 Faneuil Hall Market Pl,US,Boston,United States,NaN,217,"[206 Faneuil Hall Market Pl, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35993714563105...",42.359937,-71.055677,02109,MA,40b28c80f964a520b5f71ee3
1,Dog n' Claw,Seafood Restaurant,1 Central Wharf Boston,US,Boston,United States,NaN,730,"[1 Central Wharf Boston, Boston, MA 02110]","[{'label': 'display', 'lat': 42.35917282104492...",42.359173,-71.049538,02110,MA,4c335e83ac0ab7136bbb1c1e
2,My Busy Dog,Business Service,8 Faneuil Hall Market Pl Fl 3,US,Boston,United States,NaN,62,"[8 Faneuil Hall Market Pl Fl 3, Boston, MA 02109]","[{'label': 'display', 'lat': 42.35988214932819...",42.359882,-71.058862,02109,MA,59362c81d0a14958ac92bd26
3,Joe Wex Dog Recreation Space,Dog Run,1277 Washington Street,US,Boston,United States,East Berkeley,2104,"[1277 Washington Street (East Berkeley), Bosto...","[{'label': 'display', 'lat': 42.34269554511267...",42.342696,-71.067776,02118,MA,4bc7176114d79521e3ad66e9
4,Polka Dog Bakery,Pet Store,NaN,US,Boston,United States,NaN,363,"[Boston, MA]","[{'label': 'display', 'lat': 42.363006, 'lng':...",42.363006,-71.055918,NaN,MA,585c1342d98f36440558fe33


#### Create a map

In [192]:
venues_map_dog = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_dog.lat, dataframe_filtered_dog.lng, dataframe_filtered_dog.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_dog)

# display map
venues_map_dog

#### Search for entries containing 'Outdoors'

In [193]:
# SEARCHING FOR OUTDOORS NEAR BOSTON

#search_query = ['Outdoors']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06377d81259&client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?categoryId=4d4b7105d754a06377d81259&client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&radius=7000&limit=500'

In [194]:
# GETTING THE JSON FILE

results_outdoors = requests.get(url).json()
results_outdoors

{'meta': {'code': 200, 'requestId': '601f40bee81d5a0bdb4c9441'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ba52ff7f964a52053e938e3',
    'name': 'Boston Harbor Shipyard Marina',
    'location': {'address': '256 Marginal St',
     'lat': 42.364372011738354,
     'lng': -71.03005862204594,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.364372011738354,
       'lng': -71.03005862204594}],
     'distance': 2367,
     'postalCode': '02128',
     'cc': 'US',
     'city': 'Boston',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['256 Marginal St', 'Boston, MA 02128']},
    'categories': [{'id': '4bf58dd8d48988d1e0941735',
      'name': 'Harbor / Marina',
      'pluralName': 'Harbors / Marinas',
      'shortName': 'Harbor / Marina',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/harbor_',
       'suffix': '.png'},
      'primary': True}],
    'ref

#### Creating a dataframe

In [237]:
# assign relevant part of JSON to venues
venues_outdoors = results_outdoors['response']['venues']

# tranform venues into a dataframe
dataframe_outdoors = json_normalize(venues_outdoors)
dataframe_outdoors.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1e0941735', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ba52ff7f964a52053e938e3,256 Marginal St,US,Boston,United States,NaN,2367,"[256 Marginal St, Boston, MA 02128]","[{'label': 'display', 'lat': 42.36437201173835...",42.364372,-71.030059,NaN,02128,MA,Boston Harbor Shipyard Marina,v-1612660926,NaN
1,"[{'id': '4bf58dd8d48988d1e8941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4beff67bf831c928f56b01f2,160 Park Drive,US,Boston,United States,Joseph Lee Playground,3835,"[160 Park Drive (Joseph Lee Playground), Bosto...","[{'label': 'display', 'lat': 42.34140156426568...",42.341402,-71.097314,Fenway - Kenmore - Audubon Circle - Longwood,02115,MA,Clemente Field,v-1612660926,NaN
2,"[{'id': '4deefb944765f83613cdba6e', 'name': 'H...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b3d563ff964a520a49225e3,1 S Market St,US,Boston,United States,at Congress St & North St,157,"[1 S Market St (at Congress St & North St), Bo...","[{'label': 'display', 'lat': 42.35997750557642...",42.359978,-71.056410,Downtown Boston,02109,MA,Faneuil Hall Marketplace,v-1612660926,NaN
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",NaN,NaN,NaN,NaN,NaN,NaN,False,4283ee00f964a520dd221fe3,Tremont St,US,Boston,United States,btwn Beacon & Charles St,758,"[Tremont St (btwn Beacon & Charles St), Boston...","[{'label': 'display', 'lat': 42.355487, 'lng':...",42.355487,-71.064882,Downtown,02111,MA,Boston Common,v-1612660926,NaN
4,"[{'id': '4bf58dd8d48988d1df941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a721a76f964a5205bda1fe3,Longfellow Bridge,US,Cambridge,United States,NaN,1465,"[Longfellow Bridge, Cambridge, MA 02142]","[{'label': 'display', 'lat': 42.36146689827662...",42.361467,-71.076035,NaN,02142,MA,Longfellow Bridge,v-1612660926,NaN


In [238]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_outdoors = ['name', 'categories'] + [col for col in dataframe_outdoors.columns if col.startswith('location.')] + ['id']
dataframe_filtered_outdoors = dataframe_outdoors.loc[:, filtered_columns_outdoors]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_outdoors['categories'] = dataframe_filtered_outdoors.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_outdoors.columns = [column.split('.')[-1] for column in dataframe_filtered_outdoors.columns]

dataframe_filtered_outdoors.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Boston Harbor Shipyard Marina,Harbor / Marina,256 Marginal St,US,Boston,United States,NaN,2367,"[256 Marginal St, Boston, MA 02128]","[{'label': 'display', 'lat': 42.36437201173835...",42.364372,-71.030059,NaN,02128,MA,4ba52ff7f964a52053e938e3
1,Clemente Field,Baseball Field,160 Park Drive,US,Boston,United States,Joseph Lee Playground,3835,"[160 Park Drive (Joseph Lee Playground), Bosto...","[{'label': 'display', 'lat': 42.34140156426568...",42.341402,-71.097314,Fenway - Kenmore - Audubon Circle - Longwood,02115,MA,4beff67bf831c928f56b01f2
2,Faneuil Hall Marketplace,Historic Site,1 S Market St,US,Boston,United States,at Congress St & North St,157,"[1 S Market St (at Congress St & North St), Bo...","[{'label': 'display', 'lat': 42.35997750557642...",42.359978,-71.056410,Downtown Boston,02109,MA,4b3d563ff964a520a49225e3
3,Boston Common,Park,Tremont St,US,Boston,United States,btwn Beacon & Charles St,758,"[Tremont St (btwn Beacon & Charles St), Boston...","[{'label': 'display', 'lat': 42.355487, 'lng':...",42.355487,-71.064882,Downtown,02111,MA,4283ee00f964a520dd221fe3
4,Longfellow Bridge,Bridge,Longfellow Bridge,US,Cambridge,United States,NaN,1465,"[Longfellow Bridge, Cambridge, MA 02142]","[{'label': 'display', 'lat': 42.36146689827662...",42.361467,-71.076035,NaN,02142,MA,4a721a76f964a5205bda1fe3


#### Create a map

In [197]:
venues_map_outdoors = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_outdoors.lat, dataframe_filtered_outdoors.lng, dataframe_filtered_outdoors.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_outdoors)

# display map
venues_map_outdoors

#### Search for entries containing 'Playground'

In [216]:
# SEARCHING FOR PLAYGROUND NEAR BOSTON

search_query = ['Playground']
limit = 500
radius = 7000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit=500'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

"https://api.foursquare.com/v2/venues/search?client_id=NAF0ED02AESTUBSNE5LS34O4SBSAGDR0NX2V04Y5VJOQLTPM&client_secret=CQ5H2IZSYIMNI4B1TCYYHOCUFEJMRMBFHPHG2ICKNUH5V5FT&ll=42.3602534,-71.0582912&oauth_token=NYUYCST5B1MIDLB1Y5DS44K4ZOJMKQMX0HV2I5CP4ZSONWPX&v=20180604&query=['Playground']&radius=7000&limit=500"

In [217]:
# GETTING THE JSON FILE

results_playground = requests.get(url).json()
results_playground

{'meta': {'code': 200, 'requestId': '601f65348a07476e5f023e85'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4b6f0668f964a520b8d72ce3',
    'name': 'Edwards Playground',
    'location': {'address': 'Main St & Eden St',
     'lat': 42.37932769170876,
     'lng': -71.06838941574097,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.37932769170876,
       'lng': -71.06838941574097}],
     'distance': 2279,
     'postalCode': '02129',
     'cc': 'US',
     'city': 'Charlestown',
     'state': 'MA',
     'country': 'United States',
     'formattedAddress': ['Main St & Eden St', 'Charlestown, MA 02129']},
    'categories': [{'id': '4bf58dd8d48988d1e7941735',
      'name': 'Playground',
      'pluralName': 'Playgrounds',
      'shortName': 'Playground',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/playground_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': '

#### Create a dataframe

In [218]:
# assign relevant part of JSON to venues
venues_playground = results_playground['response']['venues']

# tranform venues into a dataframe
dataframe_playground = json_normalize(venues_playground)
dataframe_playground.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4b6f0668f964a520b8d72ce3,Main St & Eden St,US,Charlestown,United States,NaN,2279,"[Main St & Eden St, Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37932769170876...",42.379328,-71.068389,02129,MA,Edwards Playground,v-1612670261
1,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4bc5e7ad42419521b833031d,135 Prince St.,US,Boston,United States,at Snow Hill St.,664,"[135 Prince St. (at Snow Hill St.), Boston, MA...","[{'label': 'display', 'lat': 42.36619378210354...",42.366194,-71.057487,02113,MA,DeFilippo Playground (The Gassy),v-1612670261
2,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4c5ef8af17f3b71354df25e6,Union Park St.,US,Boston,United States,at Albany St,2258,"[Union Park St. (at Albany St), Boston, MA]","[{'label': 'display', 'lat': 42.342413, 'lng':...",42.342413,-71.071359,NaN,MA,Union Park St. Playground,v-1612670261
3,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,4c03f697f56c2d7f35931d66,Between Fairfield and Massachusetts Ave,US,Boston,United States,NaN,2532,"[Between Fairfield and Massachusetts Ave, Bost...","[{'label': 'display', 'lat': 42.35253746462541...",42.352537,-71.087252,NaN,MA,Miriam and Sydney Stoneman Playground,v-1612670261
4,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",False,51d971d4498ea3dab822938b,NaN,US,Cambridge,United States,NaN,2682,"[Cambridge, MA]","[{'label': 'display', 'lat': 42.367338, 'lng':...",42.367338,-71.089462,NaN,MA,Kendal Sq Garage Playground,v-1612670261


In [219]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns_playground = ['name', 'categories'] + [col for col in dataframe_playground.columns if col.startswith('location.')] + ['id']
dataframe_filtered_playground = dataframe_playground.loc[:, filtered_columns_playground]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered_playground['categories'] = dataframe_filtered_playground.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered_playground.columns = [column.split('.')[-1] for column in dataframe_filtered_playground.columns]

dataframe_filtered_playground.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Edwards Playground,Playground,Main St & Eden St,US,Charlestown,United States,NaN,2279,"[Main St & Eden St, Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37932769170876...",42.379328,-71.068389,02129,MA,4b6f0668f964a520b8d72ce3
1,DeFilippo Playground (The Gassy),Playground,135 Prince St.,US,Boston,United States,at Snow Hill St.,664,"[135 Prince St. (at Snow Hill St.), Boston, MA...","[{'label': 'display', 'lat': 42.36619378210354...",42.366194,-71.057487,02113,MA,4bc5e7ad42419521b833031d
2,Union Park St. Playground,Playground,Union Park St.,US,Boston,United States,at Albany St,2258,"[Union Park St. (at Albany St), Boston, MA]","[{'label': 'display', 'lat': 42.342413, 'lng':...",42.342413,-71.071359,NaN,MA,4c5ef8af17f3b71354df25e6
3,Miriam and Sydney Stoneman Playground,Playground,Between Fairfield and Massachusetts Ave,US,Boston,United States,NaN,2532,"[Between Fairfield and Massachusetts Ave, Bost...","[{'label': 'display', 'lat': 42.35253746462541...",42.352537,-71.087252,NaN,MA,4c03f697f56c2d7f35931d66
4,Kendal Sq Garage Playground,Playground,NaN,US,Cambridge,United States,NaN,2682,"[Cambridge, MA]","[{'label': 'display', 'lat': 42.367338, 'lng':...",42.367338,-71.089462,NaN,MA,51d971d4498ea3dab822938b


#### Create a map

In [220]:
venues_map_playground = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Boston


# add the parks as blue circle markers
for lat, lng, label in zip(dataframe_filtered_playground.lat, dataframe_filtered_playground.lng, dataframe_filtered_playground.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map_playground)

# display map
venues_map_playground

#### Drop 'neighborhood' column

In [239]:
dataframe_filtered_park = dataframe_filtered_park.drop(['neighborhood'], axis=1)
dataframe_filtered_outdoors = dataframe_filtered_outdoors.drop(['neighborhood'], axis=1)

#### Merging dataframes

In [240]:
df_compiled = dataframe_filtered_park.append([dataframe_filtered_pet, dataframe_filtered_dog, dataframe_filtered_outdoors,
                                             dataframe_filtered_playground])
df_compiled.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,"[Cross St. (btwn North & Sudbury), Boston, MA ...","[{'label': 'display', 'lat': 42.36248823184806...",42.362488,-71.056477,02113,MA,4ba50617f964a520f1d138e3
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,"[Atlantic Ave. (at Atlantic Ave.), Boston, MA ...","[{'label': 'display', 'lat': 42.36102715769776...",42.361027,-71.051451,02110,MA,4ae3580af964a520c59321e3
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,"[Boston, MA]","[{'label': 'display', 'lat': 42.36072318013166...",42.360723,-71.052873,NaN,MA,54a1d1ce498e248911805297
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,"[Charlestown, MA 02129]","[{'label': 'display', 'lat': 42.37019699121477...",42.370197,-71.061781,02129,MA,4bcf72669854d13a8f13f64d
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,"[Museum Way, Cambridge, MA 02141]","[{'label': 'display', 'lat': 42.36911761365681...",42.369118,-71.069175,02141,MA,4bd9bd3fd2cbc92844a3d1ad


#### Finding shapes of the dataframes

In [241]:
df_compiled.shape

(233, 15)

In [227]:
dataframe_filtered_park.shape

(50, 15)

In [187]:
dataframe_filtered_pet.shape

(50, 15)

In [205]:
dataframe_filtered_dog.shape

(50, 15)

In [228]:
dataframe_filtered_outdoors.shape

(33, 15)

In [215]:
dataframe_filtered_playground.shape

(50, 15)

#### Dataframe for compiled data

In [260]:
df_compiled_new = df_compiled.drop(['formattedAddress', 'labeledLatLngs', 'id'], axis=1)
df_compiled_new

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


In [274]:
df_compiled_new = df_compiled_new.reset_index(drop=True)
df_compiled_new

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


#### Choosing categories related to dogs

In [314]:
stuffIwant = ['Park', 'Dog Run', 'Playground', 'Veterinarian', 'Pet Store', 
              'Spa', 'Pet Service', 'Medical Center', 'Other Great Outdoors', 'Lake', 'Beach']
stuffIwant

['Park',
 'Dog Run',
 'Playground',
 'Veterinarian',
 'Pet Store',
 'Spa',
 'Pet Service',
 'Medical Center',
 'Other Great Outdoors',
 'Lake',
 'Beach']

#### Cleaning up dataframe

In [284]:
df_cleaned = df_compiled_new.loc[df_compiled_new['categories'].isin(stuffIwant)]
df_cleaned

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


In [286]:
df_cleaned.shape

(179, 12)

#### Final dataframe

In [287]:
df_cleaned.reset_index(drop=True)

,name,categories,address,cc,city,country,crossStreet,distance,lat,lng,postalCode,state
0,North End Park,Park,Cross St.,US,Boston,United States,btwn North & Sudbury,290,42.362488,-71.056477,02113,MA
1,Christopher Columbus Park,Park,Atlantic Ave.,US,Boston,United States,at Atlantic Ave.,569,42.361027,-71.051451,02110,MA
2,Harbor Park,Park,NaN,US,Boston,United States,NaN,448,42.360723,-71.052873,NaN,MA
3,Paul Revere Park,Park,NaN,US,Charlestown,United States,Chelsea St,1143,42.370197,-71.061781,02129,MA
4,North Point Park,Park,Museum Way,US,Cambridge,United States,NaN,1332,42.369118,-71.069175,02141,MA
5,Peters Park,Dog Run,1205 Washington,US,Boston,United States,Berkeley,2105,42.342662,-71.067686,02118,MA
6,Piers Park,Park,95 Marginal St,US,Boston,United States,at Cottage St.,1890,42.365111,-71.036263,02128,MA
7,Langone Park,Playground,542 Commercial St,US,Boston,United States,btwn Charter & Foster,952,42.368598,-71.055734,02109,MA
8,Mary O'Malley Waterfront Park,Park,Admiral's Hill,US,Chelsea,United States,NaN,3271,42.388869,-71.049211,02150,MA
9,LoPresti Park,Park,Sumner St.,US,East Boston,United States,btwn Sumner & New,1665,42.370050,-71.042984,02128,MA


#### Creating a map for dog related venues

In [297]:
dog_venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Boston


# add the venues as blue circle markers
for lat, lng, label in zip(df_cleaned.lat, df_cleaned.lng, df_cleaned.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(dog_venues_map)

# display map
dog_venues_map

#### Map for venues by category

In [325]:
colors = ['red', 'blue', 'green', 'purple', 'orange', 'beige', 'darkpurple', 'white', 'gray', 'pink', 'darkgreen']

dog_category_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Boston

for i,j in enumerate(colors):
    df_new = df_cleaned.loc[df_cleaned['categories']==stuffIwant[i]]
    for lat, lng, label in zip(df_new.lat, df_new.lng, df_new.categories): 
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=j,
        popup=label,
        fill = True,
        fill_color=j,
        fill_opacity=0.6
        ).add_to(dog_category_map)
    
dog_category_map